# Get started 
(last updated: 2024/04/17 zensvi=v0.10.2)

To use `zensvi` in a project, run the following command. Please make sure to install a compatiable version of `PyTorch` and `torchvision` separately before running the command.

See the [documentation](https://pytorch.org/get-started/locally/) for more information on how to install PyTorch.

In [ ]:
%pip install --upgrade zensvi # if zen-svi is not installed

import zensvi

print(zensvi.__version__)

# zensvi.download
A module to download Street View Images
## Mapillary

In [ ]:
from zensvi.download import MLYDownloader

mly_api_key = "YOUR_OWN_MLY_API_KEY"  # please register your own Mapillary API key at https://www.mapillary.com/dashboard/developers
# additional keyword arguments for the Mapillary API
kwarg = {
    "image_type": "all",  # The tile image_type to be obtained, either as ‘flat’, ‘pano’ (panoramic), or 'all'.
    "min_captured_at": 1484549945000,  # The min date. Format in Unix timestamp (milliseconds).
    "max_captured_at": 1642935417694,  # The max date. Format in Unix timestamp (milliseconds).
    "organization_id": [1805883732926354],  # The organization id, ID of the organization this image (or sets of images) belong to. It can be absent.
    "compass_angle": (0,180) # The compass angle of the image. It can be absent. A range of values can be provided as a tuple.
}
mly_downloader = MLYDownloader(
    mly_api_key,  # Mapillary API key
    log_path=None,  # path to the log file
)
mly_downloader.download_svi(
    "path/to/output",  # output directory
    path_pid=None,  # if you already have a list of panorama IDs, you can specify the path to the file here
    lat=None,
    lon=None,  # latitude and longitude of the location to download
    input_csv_file="",  # path to the input CSV file containing the location information
    input_shp_file="",  # path to the input shapefile containing the location information
    input_place_name="",  # name of the location to download
    id_columns=None,  # column name of the ID in the input CSV file or shapefile
    buffer=0,  # buffer size in meters around the input location
    update_pids=False,  # if True, the list of panorama IDs will be updated. If False, the list of panorama IDs will be loaded from the path_pid file
    resolution=1024,  # resolution of the image
    cropped=False,  # if True, only the upper half of the image is saved
    batch_size=1000,  # batch size for downloading images
    start_date=None,  # start date for downloading images (YYYY-MM-DD)
    end_date=None,  # end date for downloading images (YYYY-MM-DD)
    metadata_only=False,  # if True, only metadata is downloaded
    use_cache=True,  # if True, the cache is used
    **kwarg
)

## KartaView

In [ ]:
from zensvi.download import KVDownloader

kv_downloader = KVDownloader(
    log_path=None,  # path to the log file
)
kv_downloader.download_svi(
    "path/to/output_kv",  # output directory
    path_pid=None,  # if you already have a list of panorama IDs, you can specify the path to the file here
    lat=None,
    lon=None,  # latitude and longitude of the location to download
    input_csv_file="",  # path to the input CSV file containing the location information
    input_shp_file="",  # path to the input shapefile containing the location information
    input_place_name="",  # name of the location to download
    buffer=0,  # buffer size in meters around the input location
    update_pids=False,  # if True, the list of panorama IDs will be updated. If False, the list of panorama IDs will be loaded from the path_pid file
    cropped=False,  # if True, only the upper half of the image is saved
    batch_size=1000,  # batch size for downloading images
    start_date="2021-01-01",  # start date for downloading images (YYYY-MM-DD)
    end_date="2023-01-01",  # end date for downloading images (YYYY-MM-DD)
    metadata_only=False,  # if True, only metadata is downloaded
)

## Amsterdam

In [ ]:
from zensvi.download import AMSDownloader

ams_downloader = AMSDownloader(
    log_path=None,  # path to the log file
)
ams_downloader.download_svi(
    "path/to/output_ams",  # output directory
    path_pid=None,  # if you already have a list of panorama IDs, you can specify the path to the file here
    lat=None,
    lon=None,  # latitude and longitude of the location to download
    input_csv_file="",  # path to the input CSV file containing the location information
    input_shp_file="",  # path to the input shapefile containing the location information
    input_place_name="",  # name of the location to download
    buffer=0,  # buffer size in meters around the input location
    update_pids=False,  # if True, the list of panorama IDs will be updated. If False, the list of panorama IDs will be loaded from the path_pid file
    cropped=False,  # if True, only the upper half of the image is saved
    batch_size=1000,  # batch size for downloading images
    start_date="2021-01-01",  # start date for downloading images (YYYY-MM-DD)
    end_date="2023-01-01",  # end date for downloading images (YYYY-MM-DD)
    metadata_only=False,  # if True, only metadata is downloaded
)


# zensvi.metadata
A module to analyze metadata from Mapillary
## Image-level metadata analysis

List of metadata fields:
- `year`: Year of the image
- `month`: Month of the image
- `day`: Day of the image
- `hour`: Hour of the image
- `day_of_week`: Day of the week of the image
- `daytime_nighttime`: Daytime or nighttime of the image
- `season`: Season of the image
- `relative_angle`: Relative angle of the image with respect to the street
- `h3_id`: H3 ID of the image from level 0 to 15
- `speed_kmh`: Speed of the vehicle when the image was captured

In [ ]:
from zensvi.metadata import MLYMetadata

path_input = "path/to/input"
mly_metadata = MLYMetadata(path_input)
mly_metadata.compute_metadata(
    unit="image", # unit of the metadata
    indicator_list="all", # list of indicators to compute. You can specify a list of indicators in space-separated format, e.g., "year month day" or "all" to compute all indicators
    path_output="path/to/output" # path to the output file
)

## Street-level and grid-level metadata analysis

List of metadata fields:
- `coverage`: Coverage of street view images. For street-level metadata, it is the coverage of the street in terms of length covered by user-defined buffers from street view images. For grid-level metadata, it is the coverage of the grid in terms of area covered by user-defined buffers from street view images.
- `count`: Count of the street view images. For street-level metadata, it is the count of the street view images close to the street. For grid-level metadata, it is the count of the street view images in the grid.
- `days_elapsed`: Number of days elapsed between the oldest and most recent street view images. 
- `most_recent_date`: Most recent date of the street view images.
- `oldest_date`: Oldest date of the street view images.
- `number_of_years`: Number of unique years of the street view images.
- `number_of_months`: Number of unique months of the street view images.
- `number_of_days`: Number of unique days of the street view images.
- `number_of_hours`: Number of unique hours of the street view images.
- `number_of_days_of_week`: Number of unique days of the week of the street view images.
- `number_of_daytime`: Number of daytime street view images. This is computed based on the sunrise and sunset times of the location.
- `number_of_nighttime`: Number of nighttime street view images. This is computed based on the sunrise and sunset times of the location.
- `number_of_spring`: Number of spring street view images. This is computed based on the season of the location.
- `number_of_summer`: Number of summer street view images. This is computed based on the season of the location.
- `number_of_autumn`: Number of autumn street view images. This is computed based on the season of the location.
- `number_of_winter`: Number of winter street view images. This is computed based on the season of the location.
- `average_compass_angle`: Average compass angle of the street view images.
- `average_relative_angle`: Average relative angle of the street view images.
- `average_is_pano`: Average ratio of panoramic street view images.
- `number_of_users`: Number of unique users of the street view images.
- `number_of_sequences`: Number of unique sequences of the street view images.
- `number_of_organizations`: Number of unique organizations of the street view images.
- `average_speed_kmh`: Average speed of camera when the street view images were captured.

### Street-level metadata analysis

In [ ]:
path_input = "path/to/input"
mly_metadata = MLYMetadata(path_input)
mly_metadata.compute_metadata(
    unit="street",  # unit of the metadata
    indicator_list="all",  # list of indicators to compute. You can specify a list of indicators in space-separated format, e.g., "coverage count days_elapsed" or "all" to compute all indicators
    coverage_buffer=50,  # buffer size in meters for computing coverage
    path_output="path/to/output",  # path to the output file
)

### Grid-level metadata analysis

In [ ]:
path_input = "path/to/input"
mly_metadata = MLYMetadata(path_input)
mly_metadata.compute_metadata(
    unit="grid",  # unit of the metadata
    grid_resolution=7,  # resolution of the grid in terms of H3 resolution (0-15) to aggregate the metadata
    indicator_list="all",  # list of indicators to compute. You can specify a list of indicators in space-separated format, e.g., "coverage count days_elapsed" or "all" to compute all indicators
    coverage_buffer=50,  # buffer size in meters for computing coverage
    path_output="path/to/output",  # path to the output file
)

# zensvi.cv
A module to run computer vision models
## Semantic/panoptic segmentation
- Semantic segmentation: Assigns a class to each pixel in the image.
- Panoptic segmentation: Assigns a class to each pixel in the image and assigns an instance ID to each object.

For the models, we used the following pre-trained models from [Mask2Former](https://github.com/facebookresearch/Mask2Former) 
### Cityscapes

List of semantic segmentation classes for Cityscapes:
- `road`
- `sidewalk`
- `building`
- `wall`
- `fence`
- `pole`
- `traffic light`
- `traffic sign`
- `vegetation`
- `terrain`
- `sky`
- `person`
- `rider`
- `car`
- `truck`
- `bus`
- `train`
- `motorcycle`
- `bicycle`

In [ ]:
from zensvi.cv import Segmenter

# initialize the segmenter
segmenter = Segmenter()

# set arguments
dir_input = "path/to/input"
dir_image_output = "path/to/image_output"
dir_summary_output = "path/to/summary_output"
save_image_options = "segmented_image blend_image"  # segmented_image (colored image), blend_image (blended image)
save_format = "csv json"
segmenter.segment(
    dir_input,
    dir_image_output=dir_image_output,
    dir_summary_output=dir_summary_output,
    save_image_options=save_image_options,
    save_format=save_format,
)

## Mapillary Vistas

List of semantic segmentation classes for Mapillary Vistas:
- `Bird`
- `Ground Animal`
- `Curb`
- `Fence`
- `Guard Rail`
- `Barrier`
- `Wall`
- `Bike Lane`
- `Crosswalk - Plain`
- `Curb Cut`
- `Parking`
- `Pedestrian Area`
- `Rail Track`
- `Road`
- `Service Lane`
- `Sidewalk`
- `Bridge`
- `Building`
- `Tunnel`
- `Person`
- `Bicyclist`
- `Motorcyclist`
- `Other Rider`
- `Lane Marking - Crosswalk`
- `Lane Marking - General`
- `Mountain`
- `Sand`
- `Sky`
- `Snow`
- `Terrain`
- `Vegetation`
- `Water`
- `Banner`
- `Bench`
- `Bike Rack`
- `Billboard`
- `Catch Basin`
- `CCTV Camera`
- `Fire Hydrant`
- `Junction Box`
- `Mailbox`
- `Manhole`
- `Phone Booth` 
- `Pothole`
- `Street Light`
- `Pole`
- `Traffic Sign Frame`
- `Utility Pole`
- `Traffic Light`
- `Traffic Sign (Back)`
- `Traffic Sign (Front)`
- `Trash Can`
- `Bicycle`
- `Boat`
- `Bus`
- `Car`
- `Caravan`
- `Motorcycle`
- `On Rails`
- `Other Vehicle`
- `Trailer`
- `Truck`
- `Wheeled Slow`
- `Car Mount`
- `Ego Vehicle`

In [ ]:
# initialize the segmenter
dataset = "mapillary"  # this can be either "mapillary" or "cityscapes"
task = "panoptic"  # this can be either "semantic" or "panoptic"
segmenter = Segmenter(dataset=dataset, task=task)

# set arguments
dir_input = "path/to/input"
dir_image_output = "path/to/image_output"
dir_summary_output = "path/to/summary_output"
save_image_options = "segmented_image blend_image"
save_format = "csv json"
csv_format = "long"  # "long" or "wide"
segmenter.segment(
    dir_input,
    dir_image_output=dir_image_output,
    dir_summary_output=dir_summary_output,
    save_image_options=save_image_options,
    save_format=save_format,
    csv_format=csv_format,
)

## Image classification/regression
- Image classification: Assigns a class to the entire image.
- Image regression: Predicts a continuous value for the entire image.
### Places365
Places365 is a scene-centric database with 365 scene categories and 102 attributes. The model predicts the scene category and attributes of the image.
List of scene classes for Places365 can be found [here](https://github.com/CSAILVision/places365/blob/master/categories_places365.txt) and a list of attributes can be found [here](https://github.com/CSAILVision/places365/blob/master/labels_sunattribute.txt)

In [ ]:
from zensvi.cv import ClassifierPlaces365

# initialize the classifier
classifier = ClassifierPlaces365(
    device="cpu",  # device to use (either "cpu" or "gpu")
)

# set arguments
classifier = ClassifierPlaces365()
dir_input = "path/to/input"
dir_image_output = "path/to/image_output"
dir_summary_output = "path/to/summary_output"
csv_format = "long"  # "long" or "wide"
classifier.classify(
    dir_input,
    dir_image_output=dir_image_output,
    dir_summary_output=dir_summary_output,
    csv_format=csv_format,
)

### PlacePulse 2.0
PlacePulse 2.0 is a dataset that contains images of pairs of images and asks users to choose which image is better. The model predicts the image that is better. The original paper can be found [here](https://arxiv.org/pdf/1608.01769v2), and the dataset can be found [here](https://figshare.com/articles/dataset/Place_Pulse/11859993). Indicators of urban perception are:

- `Safety`
- `Liveliness`
- `Beauty`
- `Wealth`
- `Boringness`
- `Depressingness`

In [ ]:
from zensvi.cv import ClassifierPerception

classifier = ClassifierPerception(
    perception_study="safer", # Other options are "livelier", "wealthier", "more beautiful", "more boring", "more depressing"
    device="cpu",  # device to use (either "cpu" or "gpu")
)
dir_input = "path/to/input"
dir_summary_output = "path/to/summary_output"   
batch_size = 32
save_format = "csv json"
classifier.classify(
    dir_input,
    dir_summary_output=dir_summary_output,
    batch_size=batch_size,
    save_format=save_format,
)

### Global Streetscapes
Global Streetscapes is a dataset of street-level images with various attributes. The model predicts the attributes of the image.
#### Glare

List of glare classes:
- `True`: Glare is present in the image
- `False`: Glare is not present in the image

In [ ]:
from zensvi.cv import ClassifierGlare

classifier = ClassifierGlare()
dir_input = "path/to/input"
dir_summary_output = "path/to/summary_output"
classifier.classify(
    dir_input,
    dir_summary_output=dir_summary_output,
)

#### Lighting

- `day`: The image was taken during the day.
- `night`: The image was taken during the night.
- `dawn/dusk`: The image was taken during dawn or dusk.

In [ ]:
from zensvi.cv import ClassifierLighting

classifier = ClassifierLighting()
dir_input = "path/to/input"
dir_summary_output = "path/to/summary_output"
classifier.classify(
    dir_input,
    dir_summary_output=dir_summary_output,
)

#### Panorama

List of panorama classes:
- `True`: The image is a panorama.
- `False`: The image is not a panorama.

In [ ]:
from zensvi.cv import ClassifierPanorama

classifier = ClassifierPanorama()
dir_input = "path/to/input"
dir_summary_output = "path/to/summary_output"
classifier.classify(
    dir_input,
    dir_summary_output=dir_summary_output,
)

#### Platform

List of platform classes:
- `cycling surface`: The image shows a cycling surface.
- `driving surface`: The image shows a driving surface.
- `fields`: The image shows fields.
- `railway`: The image shows a railway.
- `tunnel`: The image shows a tunnel.
- `walking surface`: The image shows a walking surface.

In [ ]:
from zensvi.cv import ClassifierPlatform

classifier = ClassifierPlatform()
dir_input = "path/to/input"
dir_summary_output = "path/to/summary_output"
classifier.classify(
    dir_input,
    dir_summary_output=dir_summary_output,
)

#### Quality

List of quality classes:
- `good`: The image quality is good.
- `slightly poor`: The image quality is slightly poor.
- `very poor`: The image quality is very poor.

In [ ]:
from zensvi.cv import ClassifierQuality

classifier = ClassifierQuality()
dir_input = "path/to/input"
dir_summary_output = "path/to/summary_output"
classifier.classify(
    dir_input,
    dir_summary_output=dir_summary_output,
)

#### Reflection

List of reflection classes:
- `True`: Reflection is present in the image.
- `False`: Reflection is not present in the image.

In [ ]:
from zensvi.cv import ClassifierReflection

classifier = ClassifierReflection()
dir_input = "path/to/input"
dir_summary_output = "path/to/summary_output"
classifier.classify(
    dir_input,
    dir_summary_output=dir_summary_output,
)

#### View Direction

List of view direction classes:
-  `front/back`: The image shows the front or back view.
-  `side`: The image shows the left or right view.

In [ ]:
from zensvi.cv import ClassifierViewDirection

classifier = ClassifierViewDirection()
dir_input = "path/to/input"
dir_summary_output = "path/to/summary_output"
classifier.classify(
    dir_input,
    dir_summary_output=dir_summary_output,
)

#### Weather

List of weather classes:
- `clear`: The weather is clear.
- `cloudy`: The weather is cloudy.
- `foggy`: The weather is foggy.
- `rainy`: The weather is rainy.
- `snowy`: The weather is snowy.

In [ ]:
from zensvi.cv import ClassifierWeather

classifier = ClassifierWeather()
dir_input = "path/to/input"
dir_summary_output = "path/to/summary_output"
classifier.classify(
    dir_input,
    dir_summary_output=dir_summary_output,
)

## Depth estimation
- Depth estimation: Predicts the depth map of the image.
- Relative depth estimation: Predicts the relative depth map of the image. We used the following pre-trained models from [DPT](https://huggingface.co/Intel/dpt-large)
- Absolute depth estimation: Predicts the absolute depth map of the image. We used the following pre-trained models from [Depth Anything](https://huggingface.co/spaces/LiheYoung/Depth-Anything)

In [ ]:
from zensvi.cv import DepthEstimator

depth_estimator = DepthEstimator(
    device="cpu",  # device to use (either "cpu" or "gpu")
    task="relative" # task to perform (either "relative" or "absolute")
)

dir_input = "path/to/input"
dir_image_output = "path/to/image_output" # estimated depth map
depth_estimator.estimate_depth(
    dir_input,
    dir_image_output
)

## Low-level features
- Low-level features: Extracts low-level features from the image.
- Edge detection: Detects edges in the image.
- Blob detection: Detects blobs in the image.
- Blur detection: Detects blur in the image.
- HSL color histogram: Extracts HSL color histogram from the image.

In [ ]:
from zensvi.cv import get_low_level_features

dir_input = "path/to/input"
dir_image_output = "path/to/image_output"
dir_summary_output = "path/to/summary_output"
csv_format = "long"  # "long" or "wide"
get_low_level_features(
    dir_input,
    dir_image_output=dir_image_output,
    dir_summary_output=dir_summary_output,
    save_format="json csv",
    csv_format=csv_format,
)

## Image embeddings
- Image embeddings: Extracts embeddings from the image. We used the following packages for image embeddings: [img2vec_pytorch](https://github.com/christiansafka/img2vec)

In [ ]:
from zensvi.cv import Embeddings

emb = Embeddings(
    model_name="resnet-18", # model name to use for generating embeddings options are: 'alexnet', 'vgg-11', 'densenet', 'efficientnet_b0', 'efficientnet_b1', 'efficientnet_b2', 'efficientnet_b3', 'efficientnet_b4', 'efficientnet_b5', 'efficientnet_b6', 'efficientnet_b7'
    cuda=True) # if True, use GPU for generating embeddings
emb.generate_embedding(
    "path/to/image_directory",
    "path/to/output_directory",
    batch_size=1000,  # batch size for generating embeddings
)
results = emb.search_similar_images(
    "path/to/target_image_file",
    "path/to/embeddings_directory",
    20,  # number of similar images to retrieve
)

# zensvi.transform
A module to transform images

- Panorama to perscpective image transformation: Transforms a panorama image to a perspective image.
- Panorama to fisheye image transformation: Transforms a panorama image to a fisheye image. Types of fisheye transformations include stereographic, equidistant, equisolid, and orthographic.

In [ ]:
from zensvi.transform import ImageTransformer

dir_input = "path/to/input"
dir_output = "path/to/output"
image_transformer = ImageTransformer(dir_input=dir_input, dir_output=dir_output)
image_transformer.transform_images(
    style_list="perspective equidistant_fisheye orthographic_fisheye stereographic_fisheye equisolid_fisheye",  # list of projection styles in the form of a string separated by a space
    FOV=90,  # field of view
    theta=120,  # angle of view (horizontal)
    phi=0,  # angle of view (vertical)
    aspects=(9, 16),  # aspect ratio
    show_size=100, # size of the image to show (i.e. scale factor)
    use_upper_half=False, # if True, only the upper half of the image is used for transformation. Use this for fisheye images to estimate sky view.
) 


- Image to Point Cloud transformation: Converts depth and color images into 3D point clouds.
- Point Cloud Saving: Supports saving point clouds in multiple formats such as PCD, PLY, NumPy, and CSV.
- Point Cloud Visualization: Provides visualization tools for inspecting generated point clouds.

In [ ]:
from zensvi.transform import PointCloudProcessor
import pandas as pd

# Directories for input and output
dir_input = "path/to/input"
dir_output = "path/to/output"

# Metadata of SVI, controlling the  global attributes of generated point clouds (e.g., ID of images to process, global coordinates, and headings)
data = pd.read_csv(f"{dir_input}/point_cloud_test_df.csv")

# Initialize the PointCloudProcessor with paths to the image and depth folders
image_folder = f"{dir_input}/color"
depth_folder = f"{dir_input}/depth"
point_cloud_processor = PointCloudProcessor(image_folder=image_folder, depth_folder=depth_folder)

# Process multiple point clouds
point_cloud_processor.process_multiple_images(
    data=data,
    save_option=True,  # Save the point clouds to the specified output directory
    output_dir=dir_output,
    save_format="pcd"  # Format to save the point clouds ('pcd', 'ply', 'npz', 'csv')
)

# Optional: Visualize one of the generated point clouds
point_clouds = point_cloud_processor.process_multiple_images(data=data, save_option=False)
point_cloud_processor.visualize_point_cloud(point_clouds[0])

# Optional: Save the first generated point cloud in additional formats
point_cloud_processor.save_point_cloud_numpy(point_clouds[0], f"{dir_output}/point_cloud_0001.npz")
point_cloud_processor.save_point_cloud_csv(point_clouds[0], f"{dir_output}/point_cloud_0001.csv")

# How to visualize the results
## Point map

In [ ]:
from zensvi.visualization import plot_map

dir_input = "path/to/input"
path_output = "path/to/output.png"  # output file path
path_pid = "path/to/pid"  # path to the panorama ID file with latitude and longitude
csv_file_pattern = "pixel_ratios.csv"  # pattern of the CSV files that contain the pixel ratios (or any other variable to plot)
variable = "vegetation"  # variable to plot (e.g. vegetation, building, sky, etc.). This should be the column name in the CSV file. If None, count of the number of images is plotted
plot_type = "point"  # plot type (either "point", "line", or "hexagon")
fig, ax = plot_map(
    path_pid,
    dir_input=dir_input,
    csv_file_pattern=csv_file_pattern,
    variable_name=variable,
    plot_type=plot_type,
    path_output=path_output,
    resolution=13,
    cmap="viridis",
    legend=True,
    title="Point Map",
    legend_title="Vegetation",
    dark_mode=False,
)

## Line map

In [ ]:
dir_input = "path/to/input"
path_output = "path/to/output.png"  # output file path
path_pid = "path/to/pid"  # path to the panorama ID file with latitude and longitude
csv_file_pattern = "pixel_ratios.csv"  # pattern of the CSV files that contain the pixel ratios (or any other variable to plot)
variable = "vegetation"  # variable to plot (e.g. vegetation, building, sky, etc.). This should be the column name in the CSV file. If None, count of the number of images is plotted
plot_type = "line"  # plot type (either "point", "line", or "hexagon")
fig, ax = plot_map(
    path_pid,
    dir_input=dir_input,
    csv_file_pattern=csv_file_pattern,
    variable_name=variable,
    plot_type=plot_type,
    path_output=path_output,
    resolution=13,
    cmap="viridis",
    legend=True,
    title="Point Map",
    legend_title="Vegetation",
    dark_mode=False,
)

## Hexagon map

In [ ]:
dir_input = "path/to/input"
path_output = "path/to/output.png"  # output file path
path_pid = "path/to/pid"  # path to the panorama ID file with latitude and longitude
csv_file_pattern = "pixel_ratios.csv"  # pattern of the CSV files that contain the pixel ratios (or any other variable to plot)
variable = "vegetation"  # variable to plot (e.g. vegetation, building, sky, etc.). This should be the column name in the CSV file. If None, count of the number of images is plotted
plot_type = "hexagon"  # plot type (either "point", "line", or "hexagon")
fig, ax = plot_map(
    path_pid,
    dir_input=dir_input,
    csv_file_pattern=csv_file_pattern,
    variable_name=variable,
    plot_type=plot_type,
    path_output=path_output,
    resolution=13,
    cmap="viridis",
    legend=True,
    title="Point Map",
    legend_title="Vegetation",
    dark_mode=False,
)

## Plot images as grid

In [ ]:
from zensvi.visualization import plot_image

dir_image_input = "path/to/input"
path_output = "path/to/output.png"  # output file path
image_file_pattern = "*.png"  # pattern of the image files to plot
dir_csv_input = "path/to/csv"  # directory of the CSV files
csv_file_pattern = "pixel_ratios.csv"  # pattern of the CSV files
sort_by = "random"  # sort the images by either "random" or names of variables in the CSV files (e.g. "vegetation")
fig, ax = plot_image(
    dir_image_input,
    4,  # number of rows
    5,  # number of columns
    dir_csv_input=dir_csv_input,  # directory of the CSV files
    csv_file_pattern=csv_file_pattern,  # pattern of the CSV files
    sort_by=sort_by,  # sort the images by either "random" or names of variables in the CSV files (e.g. "vegetation")
    title="Image Grid",
    path_output=path_output,
    dark_mode=False,  # if True, the background is dark
    random_seed=123,
)

## Plot variables as Kernel Density Estimation

In [ ]:
from zensvi.visualization import plot_kde

path_input = "path/to/input.csv" # input CSV file
columns = ["vegetation", "building", "sky"] # list of columns to plot
path_output = "path/to/output.png"  # output file path
kwargs = {
    "clip": (0, 1),  # clip the values
    "palette": "twilight",  # color palette. This can be any color palette from the seaborn library or matplotlib library or your own color palette
}

plot_kde(
    path_input,
    columns,
    path_output = path_output,
    legend = True,
    title = "KDE Plot",
    legend_title = "Categories",
    dpi = 300,
    font_size = 30,
    dark_mode = False,
    **kwargs,
)

## Plot variables as histograms

In [ ]:
from zensvi.visualization import plot_hist

path_input = "path/to/input.csv" # input CSV file
columns = ["truck", "person", "car"] # list of columns to plot
path_output = "path/to/output.png"  # output file path
kwargs = {
    "clip": (0, 1),  # clip the values
    "palette": "twilight",  # color palette. This can be any color palette from the seaborn library or matplotlib library or your own color palette
}

plot_hist(
    path_input,
    columns,
    path_output = path_output,
    legend = True,
    title = "Histogram",
    legend_title = "Count",
    dpi = 300,
    font_size = 30,
    dark_mode = False,
    **kwargs,
)